## Install


In [20]:
%pip install --upgrade pip
!pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
%pip install modelscope==1.16.1
%pip install transformers --upgrade
%pip install accelerate --upgrade
%pip install peft==0.11.1
%pip install datasets==2.20.0
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Writing to /root/.config/pip/pip.conf
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/c2/60/a585c806d6c0ec5f8149d44eb202714792802f484e6e2b1bf96b23bd2b00/accelerate-1.2.1-py3-none-any.whl (336 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.32.1
    Uninstalling accelerate-0.32.1:
      Successfully uninstalled accelerate-0.32.1
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Note: you may need to restart the kernel to use update

In [1]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig

/root/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 将JSON文件转换为CSV文件
df = pd.read_json('/home/merged.json')
ds = Dataset.from_pandas(df)
ds[:3]

{'instruction': ['祝赵老师生日,文艺风格', '祝赵老师春节,文艺风格', '祝赵老师元宵节,文艺风格'],
 'input': ['', '', ''],
 'output': ['赵老师，岁月在您的身上留下了温柔的痕迹，如同五月的晨光，不愠不火，却总能温暖人心。记得第一次见您时，您正坐在窗边批改作业，阳光透过树叶的缝隙，洒在您的身上，那一刻，我仿佛看到了知识与智慧的光芒。您的每一堂课，都如木槿的清香，让人沉醉。在您的生日这天，希望时间能为您停留，让每一天都充满温馨与美好。愿您依然保持那份纯真的笑容，继续为我们点亮前行的道路。在未来的日子里，无论风雨，我们都将铭记您的教诲，勇敢地追逐梦想。感谢您，不仅教会我们知识，更教会我们如何成为更好的人。',
  '赵老师，春节之际，我总想起那些在校园里度过的时光。您曾像一盏明灯，照亮我们前行的道路，用智慧的光芒驱散未知的迷雾。记得那个冬天，您手捧热茶，耐心为我们解答难题，窗外的雪静静飘落，那一刻，时间仿佛凝固，只留下知识的种子在心底悄悄发芽。您的言传身教，如同春风拂过山野，让每一颗心都感受到了温暖与希望。在这辞旧迎新的时刻，祝愿您在新的一年里，依然能够感受到生活的美好，家庭幸福，身体健康，事业顺利。愿岁月温柔以待，愿您心中的那份热爱永远不减，继续在知识的海洋里泛舟，带领更多的人寻找到属于自己的光。',
  '在这一年的元宵佳节，我不禁回想起与您相处的点点滴滴。那些关于知识的探讨、关于生活的感悟，都如同一盏盏明灯，照亮了我前行的道路。您的话语总能像春日里的暖阳，驱散我心中的阴霾。感谢您用智慧和耐心，为我的成长播下希望的种子。在这个团圆的时刻，祝您赵老师身体健康，幸福安康，生活如这元宵节的灯笼般，明亮而温暖。']}

In [3]:
from modelscope import snapshot_download, AutoModel, AutoTokenizer
model_dir = snapshot_download('qwen/Qwen2.5-7B-Instruct', cache_dir='/home/temp', revision='master')

In [4]:
MODEL_PATH = "/home/temp/qwen/Qwen2___5-7B-Instruct"

# 处理数据集

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=False, trust_remote_code=True)
tokenizer

Qwen2Tokenizer(name_or_path='/home/temp/qwen/Qwen2___5-7B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_wor

In [6]:
def process_func(example):
    MAX_LENGTH = 1024    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

Map: 100%|██████████| 3276/3276 [00:14<00:00, 220.23 examples/s]


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3276
})

In [8]:
tokenizer.decode(tokenized_id[0]['input_ids'])

'<|im_start|>system\n你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福<|im_end|>\n<|im_start|>user\n祝赵老师生日,文艺风格<|im_end|>\n<|im_start|>assistant\n赵老师，岁月在您的身上留下了温柔的痕迹，如同五月的晨光，不愠不火，却总能温暖人心。记得第一次见您时，您正坐在窗边批改作业，阳光透过树叶的缝隙，洒在您的身上，那一刻，我仿佛看到了知识与智慧的光芒。您的每一堂课，都如木槿的清香，让人沉醉。在您的生日这天，希望时间能为您停留，让每一天都充满温馨与美好。愿您依然保持那份纯真的笑容，继续为我们点亮前行的道路。在未来的日子里，无论风雨，我们都将铭记您的教诲，勇敢地追逐梦想。感谢您，不仅教会我们知识，更教会我们如何成为更好的人。<|endoftext|>'

In [9]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"])))

'赵老师，春节之际，我总想起那些在校园里度过的时光。您曾像一盏明灯，照亮我们前行的道路，用智慧的光芒驱散未知的迷雾。记得那个冬天，您手捧热茶，耐心为我们解答难题，窗外的雪静静飘落，那一刻，时间仿佛凝固，只留下知识的种子在心底悄悄发芽。您的言传身教，如同春风拂过山野，让每一颗心都感受到了温暖与希望。在这辞旧迎新的时刻，祝愿您在新的一年里，依然能够感受到生活的美好，家庭幸福，身体健康，事业顺利。愿岁月温柔以待，愿您心中的那份热爱永远不减，继续在知识的海洋里泛舟，带领更多的人寻找到属于自己的光。<|endoftext|>'

# 创建模型

In [10]:
import torch

model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, device_map="auto",torch_dtype=torch.bfloat16)
model

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.11s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
      )
    )
    (norm):

In [11]:
model.enable_input_require_grads() # 开启梯度检查点时，要执行该方法

In [12]:
model.dtype

torch.bfloat16

# lora 

In [13]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=32, # Lora 秩
    lora_alpha=16, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=32, target_modules={'up_proj', 'gate_proj', 'q_proj', 'v_proj', 'o_proj', 'k_proj', 'down_proj'}, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [14]:
model = get_peft_model(model, config)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/home/temp/qwen/Qwen2___5-7B-Instruct', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=32, target_modules={'up_proj', 'gate_proj', 'q_proj', 'v_proj', 'o_proj', 'k_proj', 'down_proj'}, lora_alpha=16, lora_dropout=0.1, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None)

In [15]:
model.print_trainable_parameters()

trainable params: 80,740,352 || all params: 7,696,356,864 || trainable%: 1.0491


# 配置训练参数

In [16]:
args = TrainingArguments(
    output_dir="/home/output/Qwen2.5_instruct_lora2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, 
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True
)

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [18]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss
10,2.207000
20,1.915000
30,1.943400
40,1.814300
50,1.701800
60,1.705800
70,1.588000
80,1.519700
90,1.420500
100,1.461100


/root/test/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /home/temp/qwen/Qwen2___5-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/test/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /home/temp/qwen/Qwen2___5-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/test/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /home/temp/qwen/Qwen2___5-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/test/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /home/temp/qwen/Qwen2___5-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/root/test/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find

TrainOutput(global_step=612, training_loss=1.2518605264573315, metrics={'train_runtime': 1359.1261, 'train_samples_per_second': 7.231, 'train_steps_per_second': 0.45, 'total_flos': 1.8222733818803405e+17, 'train_loss': 1.2518605264573315, 'epoch': 2.9865689865689866})

# 合并加载模型

In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = MODEL_PATH
lora_path = '/home/output/Qwen2.5_instruct_lora2/checkpoint-612' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

prompt = "你是谁？"
inputs = tokenizer.apply_chat_template([{"role": "system", "content": "你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1, "temperature":0.8}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.42s/it]


我是你的智能助手，随时准备为你送上最温馨的祝福。无论是生日、节日还是其他特殊时刻，我都能为你定制专属的祝福语。


In [26]:
prompt = "祝同学腊八节,放飞自我风格"
inputs = tokenizer.apply_chat_template([{"role": "system", "content": "你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 4096, "do_sample": True, "temperature":0.6}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

同学！腊八节快乐呀！🎉 你家的腊八粥今年又熬得怎么样了？上次你发的照片，那香味儿我都快流口水了！不过你能不能别再偷偷给我寄腊八蒜了？我同事都以为我有亲戚在你家呢！😂 还有啊，你那腊八蒜泡得那么久，是不是想让我也变成个老腊八蒜？不过话说回来，你这手艺真是一绝，下次可得给我多带点！对了，你不是说要研究腊八粥的新配方吗？别忘了邀请我一起尝尝哦！最后，记得腊八节那天一定要多吃点，别光顾着工作，身体才是革命的本钱嘛！😊


In [39]:
# 保存模型和分词器
save_dir = "/home/tianji-wishes-v2.0"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


/root/test/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in /home/temp/qwen/Qwen2___5-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(


('/home/tianji-wishes-v2.0/tokenizer_config.json',
 '/home/tianji-wishes-v2.0/special_tokens_map.json',
 '/home/tianji-wishes-v2.0/vocab.json',
 '/home/tianji-wishes-v2.0/merges.txt',
 '/home/tianji-wishes-v2.0/added_tokens.json',
 '/home/tianji-wishes-v2.0/tokenizer.json')

In [40]:
model = model.merge_and_unload()

In [42]:
model.save_pretrained(save_dir)

In [43]:
tokenizer.save_pretrained(save_dir)

('/home/tianji-wishes-v2.0/tokenizer_config.json',
 '/home/tianji-wishes-v2.0/special_tokens_map.json',
 '/home/tianji-wishes-v2.0/vocab.json',
 '/home/tianji-wishes-v2.0/merges.txt',
 '/home/tianji-wishes-v2.0/added_tokens.json',
 '/home/tianji-wishes-v2.0/tokenizer.json')

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = '/home/tianji-wish2-7b' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

prompt = "祝同学腊八节,放飞自我风格"
inputs = tokenizer.apply_chat_template([{"role": "system", "content": "你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 4096, "do_sample": True, "temperature":0.7}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.19s/it]


同学！腊八节快乐呀！🎉 你家的腊八粥是不是又香又甜？上次你给我带的那碗，我到现在都念念不忘呢！你那手艺简直了，我都快爱上腊八粥了！不过，你能不能别再偷偷往里面加辣椒了？上次我喝了一大口，辣得我眼泪都出来了，结果还得你帮我解辣...😂

对了，听说你最近又在研究什么新菜谱？别忘了给我留一份哦！你那手艺，我可是期待已久啦！还有啊，你能不能别再天天发朋友圈晒美食了？我都快被你的厨艺迷住了！不过，你那个“腊八蒜”能不能也别放那么多醋？上次我吃了一个，酸得我直皱眉...😋

最后，记得腊八节那天多煮点粥，别忘了叫我一声哦！我可是你的忠实粉丝，你做的每一道菜我都在心里记着呢！哈哈，祝你腊八节快乐，身体健康，厨艺更上一层楼！🧧


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = "/home/temp/qwen/Qwen2___5-7B-Instruct" # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

prompt = "祝同学腊八节,放飞自我风格"
inputs = tokenizer.apply_chat_template([{"role": "system", "content": "你现在是一个送祝福大师，帮我针对不同人和事情、节日送对应的祝福"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 4096, "do_sample": True, "temperature":0.8}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.39s/it]


腊八节是中国的传统节日，这一天人们通常会喝腊八粥，庆祝丰收的同时也祈求来年的好运。如果你希望给你的同学送一个既契合腊八节的氛围又能鼓励他/她“放飞自我”的祝福，可以这样写：

【腊八节特别祝福】

亲爱的[同学名字]，

在这个充满喜悦与期待的腊八佳节里，愿你像这碗香甜的腊八粥一样，既有家的味道，又有生活的甜蜜。新的一年，让我们一起勇敢地拥抱变化，大胆地追求梦想，放飞自我，做最真实的自己。愿你每天都有好心情，每一步都走得精彩纷呈！

愿你在这个特别的日子里，不仅品尝到腊八粥的美味，还能感受到生活的无限可能。

祝福满满，腊八快乐！

---

这样的祝福既体现了腊八节的文化特色，又包含了鼓励和祝福的话语，非常适合送给想要鼓励的同学。
